In [1]:
# This code is written by
# Long Dinh Nguyen, PhD student at Queen's University Belfast
# Email: dinhlonghcmut@gmail.com (lnguyen04@qub.ac.uk)
#
#
#
# This convex problem is to implement a basic successive sequence program (SSP) for fractional nonlinear program (FNLP)
#
# For instance, the general FNLP problem as Energy Efficiency problem
# max    sum(rate(x(p)))/Power(x(p))
# s.t. log(1 + |h'*x(p)|^2/delta) >= r_{min}
#      ||x(p)||_2^2 <= P_{max}

# rate(x(p_i)) = log(1 + a_i*p_i/(sum_{j neq i}(a_j*p_j) + delta))
# Power(x(p)) = ||x(p)||_2^2 + P_{cir}
###########################################

In [ ]:
using Convex

# set data
k = 5; # number of elements
n = 40; # number of antennas at BS
P0 = 1000; # max power budget
P_cir = 1000;
r0 = 0.2; # data rate QoS
delta = 1e-4; # normalized Gaussian noise

# set channel model from a BS (multi-antenna) to multiple UEs (single-antenna)
H = 0.5*(randn(n,k) + im*randn(n,k))

# Assume pathloss (large-scale) channel is fixed with the value of 1e-2
path = 1e-3;

# Applying Conjugate beamforming
F_conj = H*inv(H'*H);

# Nomalized vector beamforming
f_nor = randn(n,k) + im*randn(n,k);
for i=1:k
    f_nor[:,i] = F_conj[:,i]/norm(F_conj[:,i],2);
end

beta = Array{Any,2}(zeros(k,k))
for i = 1:k
    beta[:,i] = [path*abs2(H[:,i]'*f_nor[:,i])/norm(f_nor[:,i])^2 for j = 1:k]
end
@show beta


# ##############################################
# ########## Initial point ############
p_ini = Variable(k , Positive())
term_j = Variable(0)

obj_ini = sumsquares(p_ini) + P_cir
problem = minimize(obj_ini)

# problem.constraints += sumsquares(p_ini) <= P0
for i = 1:k
    for j = 1:k
        if (j != i)
            term_j = [term_j, p_ini[j]*sqrt(beta[j,i])]
        end
    end
    # @show size(term_j)
    problem.constraints += p_ini[i]*sqrt(beta[i,i]) >= sqrt(exp(1)^r0-1)*norm([ term_j[(k-1)*(i-1)+1:(k-1)*i] ,sqrt(delta)])
# The issue has to do with the fact that a vector of variables is not equivalent to a vector-variable 
# in the current implementation of Convex.jl. Should this issue be reopened? 
end

using ECOS
@time solve!(problem, ECOSSolver())

pow_ini = p_ini.value
@show pow_ini
t_ini = sum(pow_ini.^2) + P_cir
@show t_ini

# ##############################################
# ########## Successive sequence algorithm ############

# set tau_ini
er_alg = 1; 
iter_alg = 0;
ref_sol = [];

while (er_alg >= 1e-3)

    iter_alg = iter_alg + 1;
    println("Iteration: ", iter_alg)

    if (iter_alg == 1)
        pow_ref = pow_ini;
        t_ref = t_ini;
    else
        pow_ref = pow_iter;
        t_ref = t_iter;
    end
    
    x_it = Array{Any,2}(zeros(k,1));
    a = Array{Any,2}(zeros(k,1)); b = Array{Any,2}(zeros(k,1)); c = Array{Any,2}(zeros(k,1));
    for i = 1:k
        term_x = 0;
        for j=1:k
            if(j!=i)
                term_x = term_x + beta[j,i]*pow_ref[j]^2
            end
        end
        x_it[i] = beta[i,i]*pow_ref[i]^2/(term_x+delta)
        
        a[i] = 2*log(1+x_it[i])/t_ref + x_it[i]/(t_ref*(1+x_it[i]))
        b[i] = x_it[i]^2/(t_ref*(1+x_it[i]))
        c[i] = log(1+x_it[i])/t_ref^2
    end
        
#println("!Done step!")    
    
pow = Variable(k, Positive())
t = Variable(1, Positive())    
    
obj1 = sum(a)
obj2 = sum([b[i]*delta*invpos(2*pow_ref[i]*pow[i] - pow_ref[i]^2)/beta[i,i] for i = 1:k])
obj3 = 0
    for i = 1:k
    for j = 1:k
        if(j != i)
            obj3 = obj3 + b[i]*((2*beta[j,i]*pow_ref[j]*pow[j]/(beta[i,i]*pow_ref[i]^2)) - beta[j,i]*pow_ref[j]^2*((2*pow_ref[i]*pow[i]-pow_ref[i]^2)/(beta[i]*pow_ref[i]^4)))
        end
    end
    end
obj4 = sum([c[i]*t for i = 1:k])    
    
problem = maximize(obj1 - obj2 - obj3 - obj4)

for i = 1:k
    for j = 1:k
        if (j != i)
            term_j = [term_j, pow[j]*sqrt(beta[j,i])]
        end
    end
    # @show size(term_j)
    problem.constraints += pow[i]*sqrt(beta[i,i]) >= sqrt(exp(1)^r0-1)*norm([ term_j[(k-1)*(i-1)+1:(k-1)*i] ,sqrt(delta)])
    problem.constraints += pow[i] >= pow_ref[i]/2 
# The issue has to do with the fact that a vector of variables is not equivalent to a vector-variable 
# in the current implementation of Convex.jl. Should this issue be reopened? 
end
    problem.constraints += sumsquares(pow) <= P0
    problem.constraints += t >= sumsquares(pow) + P_cir

using SCS
@time solve!(problem, SCSSolver())
    
pow_iter = pow.value
    @show pow_iter
t_iter = t.value
    @show t_iter
    
ref_sol = [ref_sol; problem.optval];
# println("ref_sol is: ", ref_sol)
if (iter_alg >= 2)
        er_alg = abs(ref_sol[iter_alg] - ref_sol[iter_alg-1])/abs(ref_sol[iter_alg-1]);
end
println("Error_Alg is: ", er_alg)
    
if (iter_alg >= 10)
    er_alg = 0;    
end
end

beta = Any[[0.129916] [0.00114132] [0.0072954] [0.000378363] [0.0035182] [0.000447217] [0.00797811] [0.00140291] [0.000946642] [0.000130478] [0.00100428] [0.000407619] [0.000357042] [0.00541068] [0.00320627] [0.0116766] [0.00587934] [0.00059821] [0.000867176] [0.00475108]; [0.00117071] [0.120374] [0.00520238] [0.00285282] [0.000491777] [0.00031868] [0.000616752] [0.00220556] [0.00107282] [0.00373769] [0.00037886] [0.00602808] [0.00924477] [0.0142152] [0.00383292] [0.0028609] [0.00818325] [0.00253991] [0.00592642] [0.00373873]; [0.0061199] [0.00425455] [0.220079] [0.00086836] [0.000176171] [0.000105503] [0.00137225] [0.000650537] [0.000365216] [0.000282122] [0.00934655] [0.00292907] [0.00181225] [0.00121114] [0.000581575] [0.00152334] [8.99237e-5] [0.000395775] [0.00120467] [0.000807603]; [0.000370231] [0.00272141] [0.00101291] [0.138666] [0.00334726] [0.00319143] [0.00170405] [0.00130568] [0.00191696] [0.00159648] [0.000509855] [0.000567402] [0.00248305] [0.00174726] [0.0050403] [0.000

  4.481748 seconds (2.88 M allocations: 2.605 GB, 36.56% gc time)
pow_iter = [0.226125; 0.933106; 1.73452; 12.305; 1.0508; 8.86945; 0.241179; 5.9899; 5.64282; 16.6137; 3.88469; 11.1006; 9.68159; 0.362662; 3.31183; 0.233357; 0.2517; 11.0744; 7.4273; 0.25388]
t_iter = 2476.6848718280016
Error_Alg is: 0.07323126436665923
Iteration: 5


  8.571829 seconds (3.73 M allocations: 4.087 GB, 40.34% gc time)
pow_iter = [0.223438; 0.874704; 2.08615; 7.92212; 1.05735; 6.71484; 0.237635; 5.23048; 6.33781; 18.2938; 3.67319; 14.5601; 12.0973; 0.341068; 2.94906; 0.230379; 0.237522; 6.80339; 7.52122; 0.25272]
t_iter = 2481.8676061923406
Error_Alg is: 0.003437111331362748
Iteration: 6


 37.693955 seconds (4.72 M allocations: 6.031 GB, 41.23% gc time)
pow_iter = [0.222768; 0.711294; 1.97944; 11.5436; 1.20519; 8.52388; 0.23548; 5.9398; 5.83071; 17.0075; 3.93713; 11.5587; 10.2046; 0.345551; 3.30176; 0.227531; 0.232703; 10.4622; 7.46612; 0.247804]
t_iter = 2486.4186485523323
Error_Alg is: 0.03132466010965673
Iteration: 7
